In [1]:
#Copyright 2019 Vraj Shah, Arun Kumar
#
#Licensed under the Apache License, Version 2.0 (the "License");
#you may not use this file except in compliance with the License.
#You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
#Unless required by applicable law or agreed to in writing, software
#distributed under the License is distributed on an "AS IS" BASIS,
#WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#See the License for the specific language governing permissions and
#limitations under the License.

from leven import levenshtein  
import pandas as pd
# from tqdm import tqdm
from sklearn.neighbors import DistanceMetric
import numpy as np 
from sklearn.metrics import accuracy_score
from statistics import mode
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import time
import editdistance
from scipy import stats

In [2]:
data = pd.read_csv('data_for_ML_num.csv')
y = data['y_act']
key_name = data['Attribute_name']
# structured_data = structured_data.drop(['y_act','Attribute_name','sample_1', 'sample_2','sample_3','sample_4','sample_5'],axis =1)
data['Num of nans'] = [float(data['Num of nans'][i])/float(data['Total_val'][i]) for i in data.index]
data['num of dist_val'] = [float(data['num of dist_val'][i])/float(data['Total_val'][i]) for i in data.index]

data1 = data[['Num of nans', 'max_val', 'mean', 'min_val', 'num of dist_val','std_dev','castability','extractability', 'len_val']]
data1 = data1.fillna(0)

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (2,5,10,11,12,13,14,15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data1 = data1.rename(columns={'mean': 'scaled_mean', 'min_val': 'scaled_min_val', 'max_val': 'scaled_max_val','std_dev': 'scaled_std_dev'})
# X_train, X_test,y_train,y_test = train_test_split(data1,y, test_size=0.2,random_state=100)


In [4]:
for index, row in data1.iterrows():
    if row['scaled_min_val'] > 10000:
        data1.iloc[index]['scaled_min_val'] = 10000
    if row['scaled_min_val'] < -10000:
        data1.iloc[index]['scaled_max_val'] = -10000 
    if row['scaled_max_val'] > 10000:
        data1.iloc[index]['scaled_max_val'] = 10000
    if row['scaled_max_val'] < -10000:
        data1.iloc[index]['scaled_max_val'] = -10000
        
# print(data1[['scaled_min_val']])
        
column_names_to_normalize = ['scaled_max_val', 'scaled_mean', 'scaled_min_val','scaled_std_dev']
# column_names_to_normalize = ['scaled_mean','scaled_std_dev', 'scaled_len_val']

x = data1[column_names_to_normalize].values
x = np.nan_to_num(x)
x_scaled = StandardScaler().fit_transform(x)
df_temp = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index = data1.index)
data1[column_names_to_normalize] = df_temp


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:9: S

In [5]:
data1 = data1.rename(columns={'Num of nans': 'Num_of_nans', 'num of dist_val': 'num_of_dist_val'})

data1.scaled_min_val = data1.scaled_min_val.astype(float)
data1.Num_of_nans = data1.Num_of_nans.astype(float)
data1.num_of_dist_val = data1.num_of_dist_val.astype(float)
data1.castability = data1.castability.astype(float)
data1.extractability = data1.extractability.astype(float)
# y.y_act = y.y_act.astype(float)

data1.to_csv('current.csv')

X_train, X_test,y_train,y_test = train_test_split(data1,y, test_size=0.2,random_state=100)
# print(X_train)
atr_train,atr_test = train_test_split(key_name, test_size=0.2,random_state=100)
# print(atr_train)

X_train.reset_index(inplace=True,drop=True)
y_train.reset_index(inplace=True,drop=True)
X_test.reset_index(inplace=True,drop=True)
y_test.reset_index(inplace=True,drop=True)
atr_train.reset_index(inplace=True,drop=True)
atr_test.reset_index(inplace=True,drop=True)

In [6]:
st = time.time()

Matrix = [[0 for x in range(len(X_train))] for y in range(len(X_test))]
dist_euc = DistanceMetric.get_metric('euclidean')

# for i in range(len(X_test)):
#     for j in range(len(X_train)):
#         dist = numpy.linalg.norm(a-b)
#         Matrix[i][j] = dist_euc.pairwise([data1.loc[i],data1.loc[j]])[0][1]

np_X_train = X_train.as_matrix()
np_X_test = X_test.as_matrix()

for i in range(len(X_test)):
    if i%100 == 0:
        print(i)
    a = np_X_test[i]
    for j in range(len(X_train)):
        b = np_X_train[j]
        dist = np.linalg.norm(a-b)
        Matrix[i][j] = dist
        
print(np_X_train[0])
    
print(time.time()-st)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
[ 0.00000000e+00 -2.75347976e-02 -1.60505450e-02  2.51197969e-02
  1.19569550e-04 -2.54573168e-02  1.00000000e+00  0.00000000e+00
  1.00000000e+00]
112.81501317


In [7]:
dict_label = {'Usable directly numeric':0, 'Usable with extraction':1, 'Usable with Extration': 1, 'Usable with extraction ':1, 'Usable directly categorical':2, 'Unusable':3, 'Context_specific':4, 'Usable directly categorical ':2}
y_train = [dict_label[i] for i in y_train]
y_test = [dict_label[i] for i in y_test]

dist = np.argsort(Matrix[0])[:1]
print(dist)
# print(y_train)
print(y_train[dist[0]])

[3598]
0


In [80]:
# 1-nearest neighbor on just Euclidean distance
y_pred = []
for i in range(len(X_test)):
    dist = np.argsort(Matrix[i])[:1]
    pred = y_train[dist[0]]
    y_pred.append(pred)
accuracy_score(y_test, y_pred)

0.737979470556456

In [97]:
for neighbr in range(1,11):
    y_pred = []
    for i in range(len(X_test)):
    #     print('---')
        dist = np.argsort(Matrix[i])[:neighbr]
        ys = []
        for x in dist:
            ys.append(y_train[x])
    #     print(ys)
        ho = stats.mode(ys)
    #     print(ho)
    #     print(ho[0][0])
        pred = ho[0][0]
        y_pred.append(pred)
    # print(y_pred)
    # print(y_test)
    acc = accuracy_score(y_test, y_pred)
    print(str(neighbr)+'--->'+str(acc))

1--->0.737979470556456
2--->0.7260940032414911
3--->0.7341977309562399
4--->0.7336574824419233
5--->0.7401404646137223
6--->0.7417612101566721
7--->0.7412209616423555
8--->0.7374392220421394
9--->0.7412209616423555
10--->0.7363587250135062


In [9]:
st = time.time()

Matrix_ed = [[0 for x in range(len(X_train))] for y in range(len(X_test))]
dist_euc = DistanceMetric.get_metric('euclidean')

# for i in range(len(X_test)):
#     for j in range(len(X_train)):
#         dist = numpy.linalg.norm(a-b)
#         Matrix[i][j] = dist_euc.pairwise([data1.loc[i],data1.loc[j]])[0][1]

np_X_train = atr_train.as_matrix()
np_X_test = atr_test.as_matrix()

for i in range(len(X_test)):
    if i%100 == 0:
        print(i)
    a = atr_test[i]
    for j in range(len(X_train)):
        b = atr_train[j]
        dist = editdistance.eval(a,b)
        Matrix_ed[i][j] = dist
        
print(np_X_train[0])
    
print(time.time()-st)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
q22LangProfScala
196.495404959


In [99]:
# 1-nearest neighbor on just Attribute Name Levenstein distance
y_pred = []
for i in range(len(X_test)):
    dist = np.argsort(Matrix_ed[i])[:1]
    pred = y_train[dist[0]]
    y_pred.append(pred)
accuracy_score(y_test, y_pred)

0.8022690437601296

In [98]:
for neighbr in range(1,11):
    y_pred = []
    for i in range(len(X_test)):
    #     print('---')
        dist = np.argsort(Matrix_ed[i])[:neighbr]
        ys = []
        for x in dist:
            ys.append(y_train[x])
    #     print(ys)
        ho = stats.mode(ys)
    #     print(ho)
    #     print(ho[0][0])
        pred = ho[0][0]
        y_pred.append(pred)
    # print(y_pred)
    # print(y_test)
    acc = accuracy_score(y_test, y_pred)
    print(str(neighbr)+'--->'+str(acc))    

1--->0.8022690437601296
2--->0.797947055645597
3--->0.8001080497028633
4--->0.7898433279308482
5--->0.7882225823878984
6--->0.7876823338735819
7--->0.7779578606158833
8--->0.779578606158833
9--->0.7736358725013506
10--->0.7649918962722853


In [100]:
# Majority vote using both Euclidean and Levenstein distance
for neighbr in range(1,11):
    y_pred = []
    for i in range(len(X_test)):
    #     print('---')
        dist_leven = np.argsort(Matrix_ed[i])[:neighbr]
        ys_leven = []
        for x in dist_leven:
            ys_leven.append(y_train[x])
#         ho_leven = stats.mode(ys_leven)
#         pred_leven = ho_leven[0][0]
        
        dist_euc = np.argsort(Matrix[i])[:neighbr]
#         ys_euc = []
        for x in dist_euc:
            ys_leven.append(y_train[x])
        ho_leven = stats.mode(ys_leven)
        pred_leven = ho_leven[0][0]
        
        y_pred.append(pred_leven)

    acc = accuracy_score(y_test, y_pred)
    print(str(neighbr)+'--->'+str(acc))    

1--->0.7790383576445165
2--->0.8260399783900594
3--->0.8465694219340897
4--->0.8503511615343058
5--->0.8600756347920043
6--->0.8606158833063209
7--->0.855213398163155
8--->0.8465694219340897
9--->0.8460291734197731
10--->0.8406266882766072


In [114]:
# Majority vote using both Euclidean and Levenstein distance
for neighbr in range(1,11):
    y_pred = []
    for i in range(len(X_test)):
#         print('---')
        dist_leven = np.argsort(Matrix_ed[i])[:neighbr]
        ys_leven = []
        for x in dist_leven:
            ys_leven.append(y_train[x])
#             ys_leven.append(y_train[x])
        ho_leven = stats.mode(ys_leven)
        pred_leven1 = ho_leven[0][0]
#         print(pred_leven)
        
        dist_euc = np.argsort(Matrix[i])[:neighbr]
        ys_euc = []
        for x in dist_euc:
            ys_euc.append(y_train[x])
        ho_leven = stats.mode(ys_euc)
        pred_leven2 = ho_leven[0][0]
#         print(pred_leven)
        
        merged_list = ys_leven + ys_euc
        ho_merged = stats.mode(merged_list)
        pred_leven3 = ho_merged[0][0]
        
#         print(pred_leven1,pred_leven2,pred_leven3)
        label2return = 0
        if pred_leven1 == pred_leven2:
            label2return = pred_leven2
        else:
            label2return = 5
    
        y_pred.append(label2return)

    acc = accuracy_score(y_test, y_pred)
    print(str(neighbr)+'--->'+str(acc))    

1--->0.6212857914640735
2--->0.6239870340356564
3--->0.6175040518638574
4--->0.6137223122636413
5--->0.6169638033495408
6--->0.6239870340356564


KeyboardInterrupt: 

In [12]:
Matrix_net = [[0 for x in range(len(X_train))] for y in range(len(X_test))]
alpha = [0.001,0.01,0.1,1,10,100,1000]

for alp in alpha:
    for i in range(len(Matrix)):
        for j in range(len(Matrix[i])):
            Matrix_net[i][j] = Matrix[i][j] + alp*Matrix_ed[i][j]

    for neighbr in range(1,11):
        y_pred = []
        for i in range(len(X_test)):
        #     print('---')
    #         print(Matrix_net[i])
            dist = np.argsort(Matrix_net[i])[:neighbr]
            ys = []
            for x in dist:
                ys.append(y_train[x])
            ho = stats.mode(ys)
            pred = ho[0][0]
            y_pred.append(pred)
        acc = accuracy_score(y_test, y_pred)
        print(str(neighbr)+'--->'+str(alp)+'--->'+str(acc))

1--->0.001--->0.8184764991896273
2--->0.001--->0.8060507833603457
3--->0.001--->0.8076715289032955
4--->0.001--->0.8055105348460292
5--->0.001--->0.8001080497028633
6--->0.001--->0.8022690437601296
7--->0.001--->0.8038897893030794
8--->0.001--->0.8001080497028633
9--->0.001--->0.7947055645596974
10--->0.001--->0.7941653160453809
1--->0.01--->0.8525121555915721
2--->0.01--->0.8373851971907077
3--->0.01--->0.8492706645056726
4--->0.01--->0.842247433819557
5--->0.01--->0.8363047001620746
6--->0.01--->0.8341437061048083
7--->0.01--->0.8363047001620746
8--->0.01--->0.8227984873041599
9--->0.01--->0.8109130199891951
10--->0.01--->0.8098325229605619
1--->0.1--->0.880064829821718
2--->0.1--->0.8692598595353863
3--->0.1--->0.8876283090221502
4--->0.1--->0.8741220961642355
5--->0.1--->0.8681793625067531
6--->0.1--->0.8616963803349541
7--->0.1--->0.8600756347920043
8--->0.1--->0.8514316585629389
9--->0.1--->0.8471096704484062
10--->0.1--->0.8433279308481901
1--->1--->0.8492706645056726
2--->1--->

In [13]:
Matrix_net = [[0 for x in range(len(X_train))] for y in range(len(X_test))]
alpha = [0.05,0.075,0.1,0.5,0.75]

for alp in alpha:
    for i in range(len(Matrix)):
        for j in range(len(Matrix[i])):
            Matrix_net[i][j] = Matrix[i][j] + alp*Matrix_ed[i][j]

    for neighbr in range(1,11):
        y_pred = []
        for i in range(len(X_test)):
        #     print('---')
    #         print(Matrix_net[i])
            dist = np.argsort(Matrix_net[i])[:neighbr]
            ys = []
            for x in dist:
                ys.append(y_train[x])
            ho = stats.mode(ys)
            pred = ho[0][0]
            y_pred.append(pred)
        acc = accuracy_score(y_test, y_pred)
        print(str(neighbr)+'--->'+str(alp)+'--->'+str(acc))

1--->0.05--->0.880064829821718
2--->0.05--->0.8708806050783361
3--->0.05--->0.8752025931928687
4--->0.05--->0.8687196110210696
5--->0.05--->0.864397622906537
6--->0.05--->0.8595353862776878
7--->0.05--->0.8530524041058887
8--->0.05--->0.8465694219340897
9--->0.05--->0.8379254457050244
10--->0.05--->0.8373851971907077
1--->0.075--->0.8789843327930849
2--->0.075--->0.8687196110210696
3--->0.075--->0.8822258238789843
4--->0.075--->0.8687196110210696
5--->0.075--->0.8633171258779039
6--->0.075--->0.8606158833063209
7--->0.075--->0.8595353862776878
8--->0.075--->0.8508914100486223
9--->0.075--->0.8438681793625068
10--->0.075--->0.8417071853052404
1--->0.1--->0.880064829821718
2--->0.1--->0.8692598595353863
3--->0.1--->0.8876283090221502
4--->0.1--->0.8741220961642355
5--->0.1--->0.8681793625067531
6--->0.1--->0.8616963803349541
7--->0.1--->0.8600756347920043
8--->0.1--->0.8514316585629389
9--->0.1--->0.8471096704484062
10--->0.1--->0.8433279308481901
1--->0.5--->0.8654781199351702
2--->0.5-

In [14]:
Matrix_net = [[0 for x in range(len(X_train))] for y in range(len(X_test))]
alpha = [0.001,0.01,0.1,1,10,100,1000]

for alp in alpha:
    for i in range(len(Matrix)):
        for j in range(len(Matrix[i])):
            Matrix_net[i][j] = alp*Matrix[i][j] + Matrix_ed[i][j]

    for neighbr in range(1,11):
        y_pred = []
        for i in range(len(X_test)):
        #     print('---')
    #         print(Matrix_net[i])
            dist = np.argsort(Matrix_net[i])[:neighbr]
            ys = []
            for x in dist:
                ys.append(y_train[x])
            ho = stats.mode(ys)
            pred = ho[0][0]
            y_pred.append(pred)
        acc = accuracy_score(y_test, y_pred)
        print(str(neighbr)+'--->'+str(alp)+'--->'+str(acc))

1--->0.001--->0.839546191247974
2--->0.001--->0.8373851971907077
3--->0.001--->0.8530524041058887
4--->0.001--->0.8411669367909238
5--->0.001--->0.8319827120475418
6--->0.001--->0.8227984873041599
7--->0.001--->0.8146947595894112
8--->0.001--->0.8092922744462453
9--->0.001--->0.8017287952458131
10--->0.001--->0.7974068071312804
1--->0.01--->0.839546191247974
2--->0.01--->0.8373851971907077
3--->0.01--->0.8525121555915721
4--->0.01--->0.8417071853052404
5--->0.01--->0.833063209076175
6--->0.01--->0.8233387358184765
7--->0.01--->0.8152350081037277
8--->0.01--->0.8103727714748784
9--->0.01--->0.8022690437601296
10--->0.01--->0.797947055645597
1--->0.1--->0.839546191247974
2--->0.1--->0.8379254457050244
3--->0.1--->0.855213398163155
4--->0.1--->0.8427876823338736
5--->0.1--->0.8352242031334414
6--->0.1--->0.823878984332793
7--->0.1--->0.8179362506753106
8--->0.1--->0.8119935170178282
9--->0.1--->0.804430037817396
10--->0.1--->0.7984873041599135
1--->1--->0.8492706645056726
2--->1--->0.8530